In [5]:
%%javascript
//IPython.load_extensions('calico-spell-check') //uncomment to add if not present

<IPython.core.display.Javascript object>

#OpenStreetMap Project
##Zach Farmer


##Table of Contents: 

1. [Problems Encountered in OSM Data](#Problems Encountered in OSM Data) 
    * [Brief Overview of Documents containing audited tags](#Brief Overview of Documents containing audited tags)
    * [Abbreviated Street Names](#Abbreviated Street Names)
    * [Incorrect and Inconsistent Postcodes](#Incorrect and Inconsistent Postcodes)
    * [Incorrect State Abbreviations](#Incorrect State Abbreviations)
    * [Incorrect City Names](#Incorrect City Names)
2. [Overview of The Data](#Overview of The Data)
    * [Sizes](#Sizes)
    * [Number of Ways and Nodes](#Number of Ways and Nodes)
    * [Uniques](#Uniques)
    * [Exploring the Data](#Exploring the Data)
3. [Further Thoughts on the dataset and OSM data collection methods](#Further Thoughts on the dataset and OSM data collection methods)


###**Map Area:** Seattle-East-Side, WA. specifically Bellevue, Kirkland, Redmond, Mercer Island, Issaquah, Sammamish   
![Image of project_area](openstreetmap_project_area.png)    

OpenStreetMap data for the this project including the above map can be exported from here: [https://www.openstreetmap.org/export#map=10/47.5937/-122.0931](https://www.openstreetmap.org/export#map=10/47.5937/-122.0931)   

OpenStreetMap is an open source mapping service, more information about their service can be found on their [about page.](https://www.openstreetmap.org/about) The specific data used for this project was downloaded using the OpenStreetMap Overpass API with the following bounds. 

I choose this particular region because I lived around here while attending university, In addition there are major technology companies in this region, i.e. Microsoft, INRIX, Google, Amazon, Expedia, etc. These companies employ many tech savvy individuals, it was my belief that this area might be well documented with meta-data as a result of some many people living in the area who understand the value of rich meta-data.

```<bounds minlat="47.5024" minlon="-122.256" maxlat="47.7144" maxlon="-121.974"/>```   

> More detailed information can be found on the returned XML data from the API call at the [openstreetmap wiki.](https://wiki.openstreetmap.org/wiki/OSM_XML)   

<a id='Problems Encountered in OSM Data'></a>

##1. Problems Encountered in OSM Data
***

Utilizing lesson 6 auditing procedures as a guide I created my own provisional auditing code process and analyzed the OSM data for Seattle WA, USA. More specifically the East-side of the greater Seattle area and discovered roughly 725,000 entries containing 460 unique tag key/value pairs for OSM data primitives 'ways' and 'nodes'. Similar to the lesson 6 auditing challenges I focused my attention on the address related tags to perform a more in-depth analysis and auditing. In this data set 'addr'(address) tags along with 149 other unique tags were hierarchical, specifically the 'parent' tag 'addr' contained further components. The 'addr' tags included 10 'child' tags related to addresses, I specifically audited:   
* addr:street  
* addr:city   
* addr:postcode  
* addr:state    

tags while leaving the rest alone. during the course of this audit I found several issues with the data, I would posit that similar errors would likely be found in other tags within the dataset and would suggest that caution be used when utilizing user entered data outside of geo-location(GPS) data without auditing beforehand.

The key issues I focused on in this audit were abbreviated street addresses, inconsistent and invalid postcode values, Invalid state name abbreviations, and invalid city names. Most of the errors outside of the street addresses and postcodes were a result of including extra data or the wrong data in a tag that belonged to a different tag.   


<a id='Brief Overview of Documents containing audited tags'></a>

###Brief Overview of Documents containing audited tags
***   

Given that there are 724,891 documents in this mongoDB I am shocked to find so few (4.3%) of the documents containing any type of address related meta-data, considering that most places in and around the city are likely to have addresses. This suggests to me that priority has been placed on the GPS location data first and secondary meta-data is only inserted by especially motivated or enthusiastic users.

####Total Number of mongodb documents for selected OSM area: 724,891    
```python
db.osm_data.find().count()
```  

####Number of mongodb documents containing address data: 31,670    
```python
db.osm_data.find({"address":{"$exists": True } } ).count()   
```   

####Number of mongodb documents containing street address data: 30,943
```python   
db.osm_data.find({"address.street":{"$exists": True } } ).count()
```

####Number of mongodb documents containing city names data: 30,215
```python
db.osm_data.find({"address.city":{"$exists":True}}).count()
```   

####Number of mongodb documents containing postcode data: 30,354
```python
db.osm_data.find({"address.postcode":{"$exists":True}}).count()
``` 

####Number of mongodb documents containing state abbrv. data: 510
```python
db.osm_data.find({"address.state":{"$exists":True}}).count()
``` 



<a id='Abbreviated Street Names'></a>

###Abbreviated Street Names
***

####Number of unique address-street tags with over-abbreviated street names: *158*
> Results of provisional python auditing code

Original | Corrected
---------|-----------
 102nd Ave SE | 102nd Avenue South-East
 105th Avenue NE | 105th Avenue North-East
 106th Ave NE | 106th Avenue North-East
 106th St | 106th Street
 107th Avene NE | 107th Avene North-East
 ...  | ...

After running my provisional python auditing code I discovered a number of abbreviations in the addr:street tag values. Above is an shortened example of the types of street name abbreviations and the fixes implemented to correct them. Considering that there existed almost 31,000 address street values and my auditing code found 158 abbreviated unique names there is only a small number of abbreviated street names. It seems likely that this data was already cleaned or that it was entered after OSM issued some guidelines on data entry as it pertained to street addresses.

####Number of addresses with street tags: *30,943*


####Top 5 contributors by user name of Street Addresses
```python
db.osm_data.aggregate([
                {"$match":{"address.street":{"$exists":1}}},\
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 5}
                 ])
```

####Results: 
1. {user: Glassman_Import, count: 19,069}     
2. {user: sctrojan79-import, count: 5,220}     
3. {user: seattlefyi_import, count: 2,185}    
4. {user: Geodesy99, count: 693}    
5. {user: bryceco, count: 627}   

It appears that when it comes to the tags related to addresses there are only a few major contributors who contributed meta-data(top 5: 27,794 of 30,943). Further analysis will find that most of these addresses reside in Kirkland WA. 

####Reported cities which contain street address information  
```python
db.osm_data.aggregate([
                {"$match":{"address.street":{"$exists":1}}},\
                {"$group":{"_id":"$address.city", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                 ])
```   

####Results:   
* {city: kirkland, count: 27,881}   
* {city: seattle, count: 1,059}   
* {city: None, count: 1,024}    
* {city: bellevue, count: 383}    
* {city: redmond, count: 328}     
* {city: hunts point, count: 192}    
* {city: sammamish, count: 33}     
* {city: issaquah, count: 26}    
* {city: mercer island, count: 7}   
* {city: newcastle, count: 4}    
* {city: renton, count: 2}    
* {city: clyde hill, count: 1}   
* {city: belevue, count: 1}   
* {city: lynwood, count: 1}   
* {city: kirkalnd, count: 1}   

These results reaffirm my observations that much of the meta-data contributed through tags that are address related have been contributed by a relatively small number of individuals for a relatively small physical area. 


<a id='Incorrect and Inconsistent Postcodes'></a>

###Incorrect and Inconsistent Postcodes
***

####Number of documents with address-postcodes: *30,354*      
####Number of bad Postcodes: *16*      
####Example of bad or invalid postcodes: 
*[['W Lake Sammamish Pkwy NE'], ['98004-4452'], ['98004-5002']]*      
  
There were almost as many postcodes recorded as street address, the 'error' rate was much lower then for street addresses. As nearly every recorded postcode was the standard 5 digit zip code with only a couple of entries containing the 4 digit extension. I aggregated up to the least common denominator and removed all digits beyond the 5-digit zip. Finally where there were street addresses present in the postcode tags I replaced them with the placeholder 'FIXME'. 

> Note: it might be a good idea for OSM to implement field frameworks for inserting meta-data for common fields, such as postcodes. A component of these frameworks would be to audit at time and point of entry, implementing basic test functions in order to prevent things like street addresses accidentally being placed in the postcode tags.

####Top 5 reported Post codes 
```python
db.osm_data.aggregate([{"$match":{"address.postcode":{"$exists":1}}},\
                {"$group":{"_id":"$address.postcode", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit" : 5}
                ])
```  

####Results:   
1. {Postcode: 98033, count: 18,982}   
2. {Postcode: 98034, count: 9,011}    
3. {Postcode: 98178, count: 775}    
4. {Postcode: 98004, count: 561}    
5. {Postcode: 98052, count: 276}    


Interestingly of the just over 30,000 reported postcodes over half of them 18,982 are for a postcode within the city of Kirkland, WA (98033). This postcode area accounts for only a small percentage of the total area looked at, and a small percentage of the population of the total observed area.    
<img src="98033_area_code.png">    
If we look at all of the reported postcodes and the city for which they were tagged, we see an even greater concentration of postcodes being reported for the city of Kirkland WA, then for any of the other cities in the observed area.    

####List of city name where postcode information was also posted  
```python
db.osm_data.aggregate([{"$match":{"address.postcode":{"$exists":1}}},\
                {"$group":{"_id":"$address.city", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                ])
```  
####Results from query: 
* {city: kirkland, count: 27,972}   
* {city: seattle, count: 1,061} # only a sliver of the city, captured accidentally in the OSM data set  
* {city: bellevue, count: 392}   
* {city: None, count: 343}   # Instances where postcode provided but the city name was not    
* {city: redmond, count: 325}   
* {city: hunts point, count: 193}   
* {city: sammamish, count: 33}   
* {city: issaquah, count: 20}   
* {city: mercer island, count: 7}   
* {city: newcastle, count: 3}   
* {city: clyde hill, count: 1}   
* {city: renton, count: 1}     
* {city: belevue, count: 1}  # Misspelled    
* {city: lynwood, count: 1}        
* {city: kirkalnd, count: 1} # Misspelled        

If we look at all of the cities that were reported, also just over 30,000, we will find that if the postcode has been provided then it is likely the city name was also provided, or visa versa. 

####List of city names reported and the their respective counts  
```python
db.osm_data.aggregate([{"$match":{"address.city":{"$exists":1}}},\
                {"$group":{"_id":"$address.city", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                ])
```    
####Results from query:  
* {city: kirkland, count: 28,116}   
* {city: seattle, count: 1,066} # only a sliver of the city, captured accidentally in the OSM data set  
* {city: bellevue, count: 432}   
* {city: redmond, count: 330}   
* {city: hunts point, count: 194}   
* {city: sammamish, count: 33}   
* {city: issaquah, count: 26}   
* {city: mercer island, count: 7}   
* {city: newcastle, count: 5}   
* {city: renton, count: 2}   
* {city: clyde hill, count: 1}   
* {city: belevue, count: 1}   # Misspelled     
* {city: lynwood, count: 1}     
* {city: kirkalnd, count: 1}  # Misspelled     

> If we are concerned with encouraging users to enter more meta-data then we should take note of the fact that users who entered city names are very likely to also enter postcodes. This fact could be leveraged to increase contributions to meta-data.    

####Top Five postcode contributors to Kirkland by user name and count 
```python
db.osm_data.aggregate([{"$match":{"address.postcode":{"$exists":1},\
                "address.city": "kirkland"}},\
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 5} ])
```

####Results:   
1. {user: Glassman_Import, count: 19,123}    
2. {user: sctrojan79-import, count: 5,243}    
3. {user: seattlefyi_import, count: 2,190}    
4. {user: sctrojan79, count: 412}    
5. {user: Debbie Bull, count: 402}      


It looks as if the vast majority of submitted meta-data regarding the postcodes came from just a couple of sources (Glassman_import, sctrojan79-import,seattlefyi_import), which all appear to be imports into OSM, possibly from large databases containing geographic related data. If a goal of OSM is to encourage all its users to contribute meta-data they should be looking into methods of encouraging 'average' users to include more meta-data with their submissions. 


<a id='Incorrect State Abbreviations'></a>

###Incorrect State Abbreviations
***

####Number of documents with address-State abbrv.: *510*      
####Number of bad or invalid state abbreviations: *3*    
####Example of bad or invalid state abbreviations:    
*[['NE 15th Street'], ['156th Avenue NE'], ['NE 18th Street']]*   

While only a small number of documents that contained addresses included the the state abbreviations I ran a simply audit on these tags in order to insure only two characters and found that once again like the postcodes, street addresses had been entered by accident or confusion into this field. Given that the user is submitting geographic data the state, country, postcode and etc. could easily be inferred by location. OSM could automatically fill out this information when a submission is made.

I suspect that for those user who do include meta-data many naturally assume that the state designation would be obvious and therefore don't include it with the rest of their meta-data. 



<a id='Incorrect City Names'></a>

###Incorrect City Names
***

####Number of documents with city names: *30,215*    
####Number of bad or invalid city names: *31*      
####Example of bad or invalid city name:
*[['kirkland,wa'], ['Bellevue, WA'], ['Bellevue, WA']]*

Finally I choose to audit the city names field which was almost as well documented as street and postcodes in documents where any address data was recorded. I found again only a few mistakes present, consisting mainly of including the state abbreviations right after the city name, I found and removed those state abbreviations. This should be an easily solvable problem by implementing some basic auditing procedures at time of entry to prevent anything longer then 2 characters. 

####Top five contributors to city names   
```python
db.osm.aggregate([{"$match":{"address.city":{"$exists":1}}},\
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 5} ]) 
```

####Results:    
1. {city: Glassman_Import, count: 19,221}    
2. {city: sctrojan79-import, count: 5,268}    
3. {city: seattlefyi_import, count: 2,204}     
4. {city: Geodesy99, u'count': 697}    
5. {city: bryceco, u'count': 549}    

> We see a close relationship again between users who provide postcode data providing city name data as well.   



<a id='Overview of The Data'></a>

##3. Overview of The Data
***

<a id='Sizes'></a>

###Sizes
***
####Size of the OSM data file for the Seattle-WA-Eastside-Region US:    *141MB * 

####Size of the OSM.json data file for the Seattle-WA-Eastside-Region US:    *148MB * 

####Size of the Mongo DB for the OSM data:  *453MB*


<a id='Number of Ways and Nodes'></a>

###Number of type Ways and Nodes
***
####Number of documents of type node: *657,718*

####Number of documents of type way: *67,173*   

####Mongo DB Query:
```python
db.osm_data.aggregate([{"$match":{"type":{"$exists":1}}},\
                {"$group":{"_id":"$type", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                 ])
```

<a id='Uniques'></a>

###Uniques values in documents 
***

####Number of unique single-tag-keys: *278*    

####Number of unique multi-tag-keys: *150*     

####Number of Unique Users: *594*   

<a id='Exploring the Data'></a>

###Exploring the Data
***

####Total Number of mongodb documents: *724,891*    

####Top 10 Contributors by count 
```python
db.osm_data.aggregate([
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 10}
                ])
```

####Results:  
    1. {user: Glassman_Import, count: 169,296}   
    2. {user: STBrenden, count: 70,400}     
    3. {user: sctrojan79-import, count: 65,594}    
    4. {user: zephyr, count: 47,942}    
    5. {user: Extramiler, count: 34,213}   
    6. {user: csytsma, count: 31,572}    
    7. {user: Heptazane, count: 30,893}   
    8. {user: seattlefyi_import, count: 23,737}   
    9. {user: Djido, count: 22,105}   
    10. {user: Glassman, count: 19,979}  
    
Top ten contributors (1.6% of all unique users) account for 515,731 of the 724,891 documents (71%). Reinforcing the notation that most of the content and therefore value contributed to OSM is contributed by a very small percentage of all the unique contributing users. 

####Number of users having contributed 1-10 times
```python
db.osm_data.aggregate([
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$group": {"_id": "$count", "num_users":{"$sum": 1}}},\
                {"$sort":{"_id":1}},\
                {"$limit": 10} ])
```

####Results:  
* {number_contributions: 1, num_users: 109}   
* {number_contributions: 2, num_users: 51}  
* {number_contributions: 3, num_users: 32}   
* {number_contributions: 4, num_users: 25}    
* {number_contributions: 5, num_users: 20}    
* {number_contributions: 6, num_users: 10}    
* {number_contributions: 7, num_users: 14}    
* {number_contributions: 8, num_users: 9}    
* {number_contributions: 9, num_users: 9}    
* {number_contributions: 10, num_users: 10}   

Of the 594 unique users 289 ( nearly 50%) of them contributed less then ten documents each. There are 136 users who contributed over 100 documents, 44 who contributed over 1,000 and just 13 to contribute to more then 10,000 documents. Suggesting that the top contributors might not be 'normal' everyday people, but businesses whose core mission could involve the collection of geo-located data. 

####Mongo DB Query for the number of users who contributed over a certain amount(10000,1000,100) of documents    
```python
db.osm_data.aggregate([
                {"$group":{"_id":"$created.user", "count":{"$sum":1}}},\
                {"$group": {"_id": "$count", "num_users":{"$sum": 1}}},\
                {"$match":{"_id":{"$gte":100}}},\ # 1000, 10000, etc
                {"$group":{"_id":"num_users","count":{"$sum":1}}},\
                {"$sort":{"_id":1}},\
                 ])
```

The following are exploratory queries on tags with a fair number of documents containing them. Of the 460 unique tags many of them are so specialized that only a few documents contain them. We will explore some of the more common tags, they will be more informative then tags contained in only a few documents. However it is important to keep in mind that even these tags possess at most 36,922 occurrences which represents only 5% of all the documents. I would mention again that without more thorough and exhaustive meta-data, analysis such as the following have to be taken with a large grain of salt.

####Number of Building tags: *36,922*   

####Most popular buildings reported
```python
db.osm_data.aggregate([
                {"$match":{"building":{"$exists":1}}},\
                {"$group":{"_id":"$building", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 10}
                ]}
```   

####Results, slightly edited for logicalness:    
* {building: house, count: 5,107}    
* {building: apartments, count: 338}    
* {building: residential, count: 269}    
* {building: commercial, count: 240}    
* {building: detached, count: 192}   
* {building: retail, count: 64}  

####Number of Amenity tags: *2,812*      

####Top 10 amenities tagged by users
```python
db.osm_data.aggregate([
                {"$match":{"amenity":{"$exists":1}}},\
                {"$group":{"_id":"$amenity", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 10}
                ])
```

####Results: 
1. {amenity: parking, count: 984}     
2. {amenity: restaurant, count: 328}    
3. {amenity: school, count: 249}    
4. {amenity: bench, count: 116}    
5. {amenity: fast_food, count: 112}    
6. {amenity: cafe, count: 111}    
7. {amenity: bank, count: 91}     
8. {amenity: toilets, count: 86}     
9. {amenity: fuel, count: 69}     
10. {amenity: bicycle_parking, count: 66}   

####Number of leisure tags: *1,355*   

####Top 10 leisure spots
```python
db.osm_data.aggregate([
                {"$match":{"leisure":{"$exists":1}}},\
                {"$group":{"_id":"$leisure", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 10}
                ])
```

####Results:
1. {leisure: pitch, count: 484}    
2. {leisure: park, count: 340}    
3. {leisure: swimming_pool, count: 179}    
4. {leisure: playground, count: 177}    
5. {leisure: sports_centre, count: 33}    
6. {leisure: track, count: 31}     
7. {leisure: garden, count: 24}     
8. {leisure: golf_course, count: 23}    
9. {leisure: slipway, count: 14}    
10. {leisure: picnic_table, count: 10} 

####Number of sport tags: *640*   

####Most Popular Sport
```python
db.osm_data.aggregate([
                {"$match":{"sport":{"$exists":1}}},\
                {"$group":{"_id":"$sport", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 1}
                ])
```
####Result:   
1. {sport: tennis, count: 177}   

####Number of cuisine tags: *261*      

####Top 10 Popular cuisines
```python
db.osm_data.aggregate([
                {"$match":{"cuisine":{"$exists":1}}},\
                {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},\
                {"$sort":{"count": -1}},\
                {"$limit": 10}
                ])
```

####Results:  
1. {cuisine: burger, count: 28}
2. {cuisine: mexican, count: 24}
3. {cuisine: sandwich, count: 22}
4. {cuisine: pizza, count: 22}
5. {cuisine: coffee_shop, count: 21}
6. {cuisine: thai, count: 13}
7. {cuisine: chinese, count: 13}
8. {cuisine: sushi, count: 13}
9. {cuisine: american, count: 12}
10. {cuisine: japanese, count: 11}

<a id='Further Thoughts on the dataset and OSM data collection methods'></a>

##3. Further Thoughts on the dataset and OSM data collection methods
***


After auditing, inserting and reviewing this data set I am left with several thoughts. First, meta-data is often not inserted along with GPS coordinates, there were nearly 725,000 total documents but when reviewing what and how many tags are to be found in these documents the most frequent tags outside of `type` are seen less then 40,000 times. This conclusion was reaffirmed when reviewing the address tags, which suggested that most submitted GPS coordinates do not contain additional tags providing meta-data about the GPS coordinates. If we are looking to use OSM data for more then just GPS directions much more work will be necessary to provide greater depth and value. For the meta-data that I audited a great amount of it was added by several users, likely these documents were inserted in bulk from another geo-locational database.   

Second, I found that at least as it concerns address information that OSM could presumably with little effort automatically fill out zip-code, city name, state, and country information using nothing more then the submitted GPS data and an API call to a government website containing location data, which are free and easily accessible. Additionally offering perhaps a uniform  and semi-standard list of tag keys and a framework or guideline for tag values for users would likely create a much cleaner and more easily searchable database. Finally given that businesses or organizations whose mission revolves around the collection and storage of geo-located data with strong meta-data can contribute the most value to OSM, developing inducements for these types of organization to export their data to OSM seems worthwhile. Crowd sourcing geo-located data is fairly straight forward, but crowd sourcing the meta-data is much messier and clearly not as effective given the low submission rates of additional meta-data. I could be that providing a framework and some guidelines that are simple to fill out might encourage otherwise uninterested contributors to spend just a few moments more to provide meta-data about their GPS coordinates. Incentivizing this behavior would benefit the entire OSM community, making it a more valuable resource.

The OSM dataset has the potential to offer a lot of value to individuals who may otherwise be priced out of the type of information that could be contained in these datasets. If OSM were to explain the potential value of rich meta-data and how it can benefit the users of OSM maps, they might be able to achieve higher rates of meta-data submission. In terms of providing clean standard data, Implementing auditing methods after the fact will work as evidenced by my auditing process above but it would certainly be easier and likely cheaper to simply enforce certain standards and guidelines for inserting data to begin with. Providing some incentive to those users who not only contribute but also adhere to meta-data guidelines could increase the submission of clean meta-data.


In [6]:
# Apply css style to notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()